In [1]:
import os
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

plt.rcParams['figure.figsize'] = (9, 7)

import psycopg2

import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

%config IPCompleter.greedy = True
%config Completer.use_jedi = False

In [2]:
from pfs.utils.coordinates.transform import ASRDM71Transform, PfiTransform

In [3]:
import lsst.afw.image as afwImage
import lsst.afw.detection as afwDetect
import lsst.afw.display as afwDisplay

In [4]:
afwDisplay.setDefaultBackend("matplotlib")

In [5]:
def frameId(visitId, subVisitId=0):
    """Return a frameId given an visitId and optionally a subVisitId"""
    return "%06d%02d" % (visitId, subVisitId)

def getPFSC(dataId=None, date=None, visit=None, subVisit=None, threshold=None, getFileName=False, saturation=4096):
    """Return an MCS image"""
    if dataId is None:
        dataId = {}

    if date is None:
        date = dataId["date"]
    if visit is None:
        visit = dataId["visit"]
    if subVisit is None:
        subVisit = dataId["subVisit"]

    if date < "2019-08-31":
        root = f"/data/mcs/{date}"
    else:
        root = f"/data_raw/{date}/mcs"

    fileName = os.path.join(root, f"PFSC{frameId(visit, subVisit)}.fits")

    if getFileName:
        return fileName

    pfsc = afwImage.ImageF(fileName)
    pfsc = afwImage.makeMaskedImage(pfsc)

    if threshold is None:
        footprints = []
    else:
        nSat = len(afwDetect.FootprintSet(pfsc, afwDetect.Threshold(saturation), "SAT").getFootprints())
        if nSat > 0:
            print(f"{nSat} saturated spots")

        threshold += np.median(pfsc.image.array)
        footprints = afwDetect.FootprintSet(pfsc, afwDetect.Threshold(threshold), "DETECTED").getFootprints()
    
    return pfsc, footprints

def getMcsData(dataId):
    with conn:
        mcs_data = pd.read_sql(f'''
            SELECT DISTINCT 
                spot_id, mcs_center_x_pix, mcs_center_y_pix
            FROM mcs_data
            WHERE
              mcs_frame_id = {frameId(dataId["visit"], dataId["subVisit"])}
            -- limit 10
            ''', conn)
        
    return mcs_data

In [6]:
# See ~/pgpass
#   *:*:opdb:pfs:XXXX

conn = psycopg2.connect("dbname='opdb' host='pfsa-db01' port=5432 user='pfs'") 

#conn = psycopg2.connect("dbname='opdb' host='133.40.160.36' port=5432 user='pfs'") 

In [7]:
from pfs.utils.butler import Butler

butler = Butler(configRoot=os.path.join(os.environ["PFS_INSTDATA_DIR"], "data"))

# Read fiducial and spot geometry
fids = butler.get('fiducials')
dots = butler.get('black_dots')

In [8]:
import importlib
import pfs.utils.coordinates.transform
transform = importlib.reload(pfs.utils.coordinates.transform)
PfiTransform = transform.PfiTransform
ASRDM71Transform = transform.ASRDM71Transform

if True:
    dataId=dict(date="2021-09-19", visit=67962, subVisit=0)    
else:  # ask opdb for most recent exposure
    with conn:
        tmp = pd.read_sql(f'SELECT mcs_frame_id FROM mcs_data ORDER BY mcs_frame_id DESC LIMIT 1;', conn)
    frame_id = int(tmp.mcs_frame_id[0])

    visit = frame_id//100
    subVisit = frame_id%100
    
    dataId.update(visit=visit, subVisit=subVisit)

    with conn:
        t = pd.read_sql(f'SELECT taken_at, taken_in_hst_at FROM mcs_exposure WHERE mcs_frame_id = {frameId(dataId["visit"], dataId["subVisit"])};', conn)
        t = t.taken_in_hst_at[0]  # !!
    dataId.update(date=f"{t.year}-{t.month:02}-{t.day:02}")

#print("Reading %(visit)d:%(subVisit)d" % dataId)

raw = getPFSC(dataId)[0]

mcs_data = getMcsData(dataId)
mcs_data0 = mcs_data.copy(deep=True)

with conn:
    tmp = pd.read_sql(f'SELECT insrot, altitude FROM mcs_exposure where pfs_visit_id = {dataId["visit"]};', conn)
altitude, insrot = tmp.altitude[0], tmp.insrot[0]
#
# Get the transform between PFI and MCP coordinates, then transform fiducials/dots to MCS coords
#
mpt = PfiTransform(altitude=altitude, insrot=insrot)
mpt.updateTransform(mcs_data, fids)

x_fid, y_fid = mpt.pfiToMcs(fids.x_mm, fids.y_mm)

x_dot, y_dot = mpt.pfiToMcs(dots.x, dots.y)
x, y = mpt.pfiToMcs(dots.x + dots.r, dots.y)
r_dot = np.hypot(x - x_dot, y - y_dot)

In [9]:
fig = 5; plt.close(fig); fig = plt.figure(fig)
disp5 = afwDisplay.Display(fig, reopenPlot=False)

disp5.scale('asinh', 'zscale')

disp5.setMaskPlaneColor("DETECTED", "IGNORE")
disp5.setMaskPlaneColor("SAT", "IGNORE")

disp5.mtv(raw, title=f"{'date=%(date)s visit=%(visit)d subVisit=%(subVisit)d' % dataId}")
#
# Show detected dots
#
plt.plot(mcs_data.mcs_center_x_pix, mcs_data.mcs_center_y_pix, '.', alpha=0.5)
#
# And nominal positions of fiducials
#
plt.plot(x_fid, y_fid, '+', color='blue')
assert (x_fid == mpt.pfiToMcs(fids.x_mm, fids.y_mm)[0]).all()
#
# And nominal positions of black dots
#
ax = plt.gca()
for x, y, r in zip(x_dot, y_dot, r_dot):
    c = Circle((x, y), r, color='red', alpha=0.3)
    ax.add_patch(c)

<IPython.core.display.Javascript object>

In [ ]:
#
# We don't have a fiducial fibre image at the same insrot and taken around the same time as 67472, 
# so process the data ourselves to generate the "mcs_data.mcs_center_[xy]_pix" arrays
#
dataId=dict(date="2021-09-16", visit=67473, subVisit=0)  # rotation tests 

raw, fs = getPFSC(dataId, threshold=6000)

with conn:
    tmp = pd.read_sql(f'SELECT insrot, altitude FROM mcs_exposure where pfs_visit_id = {dataId["visit"]};', conn)
altitude, insrot = tmp.altitude[0], tmp.insrot[0]

x = np.empty(len(fs))
y = np.empty_like(x)

for i, foot in enumerate(fs):
    x[i], y[i] = foot.getCentroid()
    
mcs_data = pd.DataFrame(dict(mcs_center_x_pix=x, mcs_center_y_pix=y))
mcs_data0 = pd.DataFrame(dict(mcs_center_x_pix=x.copy(),
                              mcs_center_y_pix=y.copy()))
#
# And build the PfiTransform
#
mpt = PfiTransform(altitude=altitude, insrot=insrot)
mpt.updateTransform(mcs_data, fids)

In [ ]:
#
# Show an MCS image when we have the dome lights on to show PFI structures
#
from lsst.meas.algorithms import SubtractBackgroundTask

config = SubtractBackgroundTask.ConfigClass()
config.binSize = 64
backgroundTask = SubtractBackgroundTask(config=config)
#
# Specify data and read it
#
dataId=dict(date="2021-09-16", visit=67473, subVisit=0)

raw = getPFSC(dataId)[0]

# subtract an estimate of the background
backgroundTask.run(afwImage.makeExposure(raw))
# and the median column to get rid of some of the bias striping
raw.image.array -= np.median(raw.image.array, axis=0)
#
# Display time
#
fig = 1; plt.close(fig); fig = plt.figure(fig)
disp = afwDisplay.Display(fig, reopenPlot=False)

disp.scale('asinh', 0, 40, Q=2)

disp.setMaskPlaneColor("DETECTED", "IGNORE")
disp.setMaskPlaneColor("SAT", "IGNORE")
#
# Transform to MCS coordinates
#
x_dot, y_dot = mpt.pfiToMcs(dots.x, dots.y)
x, y = mpt.pfiToMcs(dots.x + dots.r, dots.y)
r_dot = np.hypot(x - x_dot, y - y_dot)

#fids = butler.get('fiducials')
x_fid, y_fid = mpt.pfiToMcs(fids.x_mm, fids.y_mm)
#
# Choose which part[s] of the PFI we want to display
#
hsize = 150   # half-size of images centered at centers
if True:     # entire field
    hsize = 2700
    centers = [(mpt.mcs_boresight_x_pix, mpt.mcs_boresight_y_pix)]
elif True:   # Show fiducials
    centers = []
    for fid in [25, 76, 56, 44]:
        l = fiducialId == fid
        centers.append((x_fid[l], y_fid[l]))
else:         # four places symmetrically around the field
    rad = 1600
    centers = [(mpt.mcs_boresight_x_pix - rad, mpt.mcs_boresight_y_pix + rad), (mpt.mcs_boresight_x_pix + rad, mpt.mcs_boresight_y_pix + rad),
               (mpt.mcs_boresight_x_pix - rad, mpt.mcs_boresight_y_pix - rad), (mpt.mcs_boresight_x_pix + rad, mpt.mcs_boresight_y_pix - rad)]

for i, (cx, cy) in enumerate(centers, 1):
    if len(centers) == 1:
        ax = plt.gca()

    elif len(centers) == 4:
        ax = fig.add_subplot(2, 2, i)
    else:
        raise RuntimeError("I only understand requests for one or four centres; sorry")

    disp.mtv(raw[cx-hsize:cx+hsize+1, cy-hsize:cy+hsize+1])
    if len(centers) > 1:
        disp.show_colorbar(False)

    x, y, r = x_dot, y_dot, r_dot

    ax = plt.gca()
    for x, y, r in zip(x, y, r):
        c = Circle((x, y), r, color='red', alpha=0.3)
        ax.add_patch(c)

    if True:
        plt.plot(x_fid, y_fid, '+', color='blue', alpha=0.6, label="fiducial")
    else:
        for x, y, fid in zip(x_fid, y_fid, fiducialId):
            plt.text(x, y, str(fid), color='blue', 
                     horizontalalignment='center', verticalalignment='center')

    plt.xlim(cx-hsize, cx+hsize+1)
    plt.ylim(cy-hsize, cy+hsize+1)

plt.suptitle(f"{'date=%(date)s visit=%(visit)d subVisit=%(subVisit)d' % dataId}\naltitude={altitude:.1f} insrot={insrot:.1f}", y=0.95);

### Now the ASRD 71M camera

## Convert ASRD geometry file from pixels to mm

In [ ]:
asrd_geom_mm = pd.read_csv("asrd_geometry_mm.csv")

cobra_x_mm = asrd_geom_mm.cobra_x_mm
cobra_y_mm = asrd_geom_mm.cobra_y_mm
cobra_L1_mm = asrd_geom_mm.cobra_L1_mm
cobra_L2_mm = asrd_geom_mm.cobra_L2_mm

# Draw L1/L2
def showCobras(theta=0, phi=90):
    # Check the definitions of theta and phi
    L1_mm = np.where(cobra_L1_mm < 100, cobra_L1_mm, np.NaN)
    L2_mm = np.where(cobra_L2_mm < 100, cobra_L2_mm, np.NaN)

    theta = np.deg2rad(theta)
    phi = np.deg2rad(phi)
    # L1
    foo = np.empty((3, len(cobra_x_mm)))
    foo[0] = cobra_x_mm
    foo[1] = foo[0] + L1_mm*np.cos(theta)
    foo[2] = np.NaN
    x = foo.T.flatten()

    foo[0] = cobra_y_mm
    foo[1] = foo[0] + L1_mm*np.sin(theta)
    foo[2] = np.NaN
    y = foo.T.flatten()

    plt.plot(*mpt.pfiToMcs(x, y), '-', color='magenta')

    # L2
    foo[0] = cobra_x_mm + L1_mm*np.cos(theta)
    foo[1] = foo[0] + L2_mm*np.cos(phi - theta)
    foo[2] = np.NaN
    x = foo.T.flatten()

    foo[0] = cobra_y_mm + L1_mm*np.sin(theta)
    foo[1] = foo[0] + L2_mm*np.sin(phi - theta)
    foo[2] = np.NaN
    y = foo.T.flatten()

    plt.plot(*mpt.pfiToMcs(x, y), '-', color='magenta');

plt.figure(1)

showCobras(theta=0, phi=90)

# Draw cobra centers
plt.plot(*mpt.pfiToMcs(cobra_x_mm, cobra_y_mm), 'o', alpha=0.5, label="cobra centre", color='chartreuse')

# and centres measured directly on the MCS
cobraRotationCenter = pd.read_csv(os.path.join(os.environ["PFS_INSTDATA_DIR"], "data/pfi/cobraRotationCenter.csv"))
plt.plot(cobraRotationCenter.x[:], cobraRotationCenter.y[:], '+', color='yellow', label="theta circle centre", zorder=1)

if False:
    disp.zoom(32, 4000, 3850)
    plt.legend();

In [ ]:
dirName = os.path.join(os.environ["PFS_INSTDATA_DIR"], "data/pfi/dot")

dotDF = pd.read_csv(os.path.join(dirName, "ff_subaru_20210710_el90_rot+00_ave.csv"))
x_fid, y_fid = dotDF.x_pixel, dotDF.y_pixel  # camera pixels

#fids = butler.get('fiducials')

mcs_data = pd.DataFrame(dict(mcs_center_x_pix=x_fid, mcs_center_y_pix=y_fid))
mcs_data0 = mcs_data.copy(deep=True)

apt = ASRDM71Transform()
apt.updateTransform(mcs_data, fids)  # not needed for now

raw = afwImage.ImageF("/home/cloomis/dots-benchcam-subaru/2021-07-10/PFSC22587900.fits")

fig = 3; plt.close(fig); fig = plt.figure(fig)
disp3 = afwDisplay.Display(fig, reopenPlot=False)
disp3.scale('asinh', 'zscale')

disp3.mtv(raw)

plt.plot(x_fid, y_fid, '+', color='blue', label="fiducial")

x, y = apt.pfiToMcs(fids.x_mm, fids.y_mm)

plt.plot(x, y, '.', alpha=0.5, color='red');

In [ ]:
# Update an old geometry in ASRD M71 coords to PFI mm
asrd_geom = pd.read_csv(os.path.join(os.environ["PFS_INSTDATA_DIR"], "data/pfi/asrd_geometry.csv"))  # cobra geometry in ASRD71M 
apt = ASRDM71Transform()

cobra_x_mm, cobra_y_mm = apt.mcsToPfi(asrd_geom.cobra_x_pixel, asrd_geom.cobra_y_pixel)

x, y = apt.mcsToPfi(asrd_geom.cobra_x_pixel + asrd_geom.cobra_L1, asrd_geom.cobra_y_pixel)
cobra_L1_mm = np.hypot(cobra_x_mm - x, cobra_y_mm - y)

x, y = apt.mcsToPfi(asrd_geom.cobra_x_pixel + asrd_geom.cobra_L2, asrd_geom.cobra_y_pixel)
cobra_L2_mm = np.hypot(cobra_x_mm - x, cobra_y_mm - y)

# ---- Repack

asrd_geom_mm = pd.DataFrame(dict(
    cobra_x_mm=cobra_x_mm, cobra_y_mm=cobra_y_mm,
    cobra_L1_mm=cobra_L1_mm, cobra_L2_mm=cobra_L2_mm))

asrd_geom_mm.to_csv("asrd_geometry_mm.csv")